In [4]:
import os
import pandas as pd
from PIL import Image

def convert_to_yolo(x_min, y_min, x_max, y_max, img_w, img_h):
    x_center = ((x_min + x_max) / 2) / img_w
    y_center = ((y_min + y_max) / 2) / img_h
    width = (x_max - x_min) / img_w
    height = (y_max - y_min) / img_h
    return round(x_center, 6), round(y_center, 6), round(width, 6), round(height, 6)

def csv_to_yolo(csv_file, img_dir, label_output_dir):
    df = pd.read_csv(csv_file)
    os.makedirs(label_output_dir, exist_ok=True)

    for image_name, group in df.groupby('filename'):
        img_path = os.path.join(img_dir, image_name)
        txt_path = os.path.join(label_output_dir, image_name.replace('.jpg', '.txt'))

        if not os.path.exists(img_path):
            print(f"Skipping {image_name}, image not found.")
            continue

        img = Image.open(img_path)
        w, h = img.size

        lines = []
        for _, row in group.iterrows():
            x_center, y_center, width, height = convert_to_yolo(
                row['xmin'], row['ymin'], row['xmax'], row['ymax'], w, h
            )
            lines.append(f"0 {x_center} {y_center} {width} {height}")  # '0' is graffiti class

        with open(txt_path, 'w') as f:
            f.write('\n'.join(lines))

# Run for train and test
csv_to_yolo('Bounding_boxes/train_labels.csv', 'images/train', 'labels/train')
csv_to_yolo('Bounding_boxes/test_labels.csv', 'images/test', 'labels/test')


In [5]:
yaml_content = """
train: ../images/train
val: ../images/test

nc: 1
names: ['graffiti']
"""

with open("graffiti.yaml", "w") as f:
    f.write(yaml_content)


In [ ]:
# 1. Change to yolov5 directory
import os
os.chdir('C:/Users/Hp/Desktop/AIE/Assessment5/yolov5')

# 2. Run training
!python train.py --img 416 --batch 8 --epochs 10 --data ../graffiti.yaml --weights yolov5s.pt --name graffiti_exp1


In [ ]:
import torch
import pandas as pd
import cv2

def compute_iou(pred_boxes, gt_boxes):
    """
    Calculate the Intersection over Union (IoU) between predicted and ground truth boxes.
    :param pred_boxes: List of [x_min, y_min, x_max, y_max] for predictions
    :param gt_boxes: List of [x_min, y_min, x_max, y_max] for ground truth
    :return: IoU value
    """
    xi1 = max(pred_boxes[0], gt_boxes[0])
    yi1 = max(pred_boxes[1], gt_boxes[1])
    xi2 = min(pred_boxes[2], gt_boxes[2])
    yi2 = min(pred_boxes[3], gt_boxes[3])
    inter_area = max(xi2 - xi1, 0) * max(yi2 - yi1, 0)

    box1_area = (pred_boxes[2] - pred_boxes[0]) * (pred_boxes[3] - pred_boxes[1])
    box2_area = (gt_boxes[2] - gt_boxes[0]) * (gt_boxes[3] - gt_boxes[1])
    union_area = box1_area + box2_area - inter_area
    return inter_area / union_area if union_area else 0

def evaluate_model_on_test(model, test_images_folder, test_labels_file):
    """
    Evaluates the YOLO model on test data, calculating IoU and saving the results to CSV.
    :param model: Trained YOLO model
    :param test_images_folder: Path to test images folder
    :param test_labels_file: Path to CSV file with test labels
    """
    df = pd.read_csv(test_labels_file)
    results = []
    
    # Loop through each image
    for _, row in df.iterrows():
        img_path = f"{test_images_folder}/{row['image_name']}"
        img = cv2.imread(img_path)
        results = model(img)
        
        # Get predictions and ground-truth boxes
        predictions = results.xyxy[0]  # Format: [x1, y1, x2, y2, confidence, class]
        gt_boxes = row[['x_min', 'y_min', 'x_max', 'y_max']].values

        # IoU Calculation (using the first detected box)
        if len(predictions) > 0:
            pred_box = predictions[0, :4].cpu().numpy()  # First prediction box
            iou = compute_iou(pred_box, gt_boxes)
            confidence = predictions[0, 4].item()  # Confidence score of the prediction
        else:
            iou = 0
            confidence = 0

        results.append([row['image_name'], confidence, iou])

    # Save results to CSV
    output_csv = "iou_results.csv"
    results_df = pd.DataFrame(results, columns=['image_name', 'confidence_value', 'IoU_value'])
    results_df.to_csv(output_csv, index=False)

    print(f"IoU results saved to {output_csv}")

# Example usage:
# Load your trained model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='runs/train/graffiti_exp1/weights/best.pt')

# Evaluate on the test dataset
evaluate_model_on_test(model, "images/test", "Bounding_boxes/test_labels.csv")


Using cache found in C:\Users\Hp/.cache\torch\hub\ultralytics_yolov5_master
